In [1]:
import pandas as pd 
import numpy as np 
from datetime import datetime 
import matplotlib.pyplot as plt 
import yfinance as yf 
import seaborn as sns 
plt.style.use('fivethirtyeight')
import random as random 

In [2]:
##import the names of the stocks in the smp500

In [3]:
sp500Names = pd.read_csv('smp500names.csv')
sp500Names = sp500Names.set_index('Symbol')
sp500Names['goldenBuySignal'] = 0 
sp500Names['bollingerBuySignal'] = 0 
sp500Names['rsiBuySignal'] =0 

In [4]:
sp500Names.tail()

,Security,GICS Sector,GICS Sub-Industttry,goldenBuySignal,bollingerBuySignal,rsiBuySignal
Symbol,,,,,,
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,0,0,0
AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",0,0,0
AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,0,0,0
AAL,American Airlines Group,Industrials,Airlines,0,0,0
A,Agilent Technologies Inc,Health Care,Health Care Equipment,0,0,0


In [5]:
##create a program that looks at the bollingerbands, Golden cross and RSI 

In [6]:
sp500Names.head()

,Security,GICS Sector,GICS Sub-Industttry,goldenBuySignal,bollingerBuySignal,rsiBuySignal
Symbol,,,,,,
ZTS,Zoetis,Health Care,Pharmaceuticals,0,0,0
ZION,Zions Bancorp,Financials,Regional Banks,0,0,0
ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,0,0,0
ZBH,Zimmer Biomet,Health Care,Health Care Equipment,0,0,0
YUM,Yum! Brands Inc,Consumer Discretionary,Restaurants,0,0,0


In [7]:
stock = yf.Ticker('WFC')

In [8]:
stockData = stock.history('300d')

In [9]:
stockData.tail(7)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-02-04,31.870001,32.849998,31.799999,32.660000,30825400,0.1,0
2021-02-05,33.000000,33.189999,32.509998,32.560001,24892700,0.0,0
2021-02-08,32.779999,33.570000,30.770000,33.400002,29223800,0.0,0
2021-02-09,33.209999,33.410000,32.910000,33.259998,23539800,0.0,0
2021-02-10,33.480000,33.689999,33.090000,33.200001,22687000,0.0,0
2021-02-11,33.200001,33.340000,32.369999,32.720001,22383400,0.0,0
2021-02-12,32.650002,33.619999,32.650002,33.529999,34449600,0.0,0


In [10]:
stock.history('200d')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-04-30,28.397686,28.678564,27.700336,28.136179,29506100,0.0,0
2020-05-01,27.603479,27.632536,26.421857,26.722105,35197600,0.0,0
2020-05-04,26.111921,26.586509,25.763247,26.489655,34343900,0.0,0
2020-05-05,26.654308,26.925499,25.424258,25.492056,56753400,0.0,0
2020-05-06,25.627652,25.811676,24.600997,24.804392,57732900,0.0,0
...,...,...,...,...,...,...,...
2021-02-08,32.779999,33.570000,30.770000,33.400002,29223800,0.0,0
2021-02-09,33.209999,33.410000,32.910000,33.259998,23539800,0.0,0
2021-02-10,33.480000,33.689999,33.090000,33.200001,22687000,0.0,0


In [11]:
name = 'WFC'

In [12]:
for name, row in sp500Names.iterrows(): 
    count=0 
    n=5
    stock = yf.Ticker(name)
    stockData = stock.history('400d')
    ##check if a stock is above its current bollingerband 
    stockData['MA20'] = stockData['Close'].rolling(window=20).mean()
    stockData['20dSTD'] = stockData['Close'].rolling(window=20).std()
    stockData['Upper'] = stockData['MA20'] + (stockData['20dSTD']*2)
    stockData['Lower'] = stockData['MA20'] - (stockData['20dSTD']*2)
    stockData['MA200'] = stockData['Close'].rolling(window=200).mean()
    stockData['MA50'] = stockData['Close'].rolling(window=50).mean()
    if stockData['Close'][-1] > stockData['Upper'][-1]:
        sp500Names.loc[(sp500Names.index == name),'bollingerBuySignal']  = 1
   # else: 
    #    print('NO') 
    ##Check what the RSI is on the given stocks 
    delta = np.diff(stockData['Close'])
    seed = delta[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(stockData['Close'])
    rsi[:n] = 100. - 100./(1.+rs)
    for i in range(n, len(stockData['Close'])):
        deltas = delta[i-1] # cause the diff is 1 shorter

        if deltas>0:
            upval = deltas
            downval = 0.
        else:
            upval = 0.
            downval = -deltas

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)
    if rsi[-1] < 35: 
        sp500Names.loc[(sp500Names.index == name), 'rsiBuySignal'] = 1 
 #   else: 
 #       print('No')
    ##now check for a goldenCross
    if stockData['MA200'][-1] < stockData['MA50'][-1]: 
        for i in range(0,7): 
            if stockData['MA200'][-i] > stockData['MA50'][-i]:
                print(stockData['MA200'][-i])
                count = count + 1 
 #           else: 
 #               print('FALSE')
 #   else: 
 #       print('NO Sir')
    if count > 0: 
        sp500Names.loc[(sp500Names.index == name), 'goldenBuySignal'] = 1 
 #   else: 
 #       print("NOPE")

C:\Users\andre\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\andre\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\andre\anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in double_scalars


52.704689540863036
52.70019496917725
52.69507703781128
52.68698701858521


In [13]:
df = sp500Names

In [14]:
df.reset_index(drop=True, inplace=True)

In [15]:
df.head(1)

,Security,GICS Sector,GICS Sub-Industttry,goldenBuySignal,bollingerBuySignal,rsiBuySignal
0,Zoetis,Health Care,Pharmaceuticals,0,0,0


In [16]:
df['totalBuy'] = df.goldenBuySignal + df.bollingerBuySignal + df.rsiBuySignal

In [17]:
df.groupby(['GICS Sector', 'Security']).agg({'totalBuy':['max']})

totalBuy
                                                                   max
GICS Sector            Security                                       
Communication Services AT&T Inc.                                     0
                       Activision Blizzard                           0
                       Alphabet Inc. (Class A)                       0
                       Alphabet Inc. (Class C)                       0
                       Charter Communications                        0
...                                                                ...
Utilities              Public Service Enterprise Group (PSEG)        0
                       Sempra Energy                                 0
                       Southern Company                              0
                       WEC Energy Group                              1
                       Xcel Energy Inc                               1

[503 rows x 1 columns]

In [18]:
df[df.bollingerBuySignal > 0 ].groupby(['GICS Sector', 'Security']).agg({'totalBuy':['max']})

totalBuy
                                                            max
GICS Sector            Security                                
Communication Services Discovery, Inc. (Class A)              1
                       Discovery, Inc. (Class C)              1
                       Live Nation Entertainment              1
                       News Corp. Class A                     1
                       News Corp. Class B                     1
                       Twitter, Inc.                          1
Consumer Discretionary Mohawk Industries                      1
Consumer Staples       Coca-Cola Company                      1
Health Care            Bio-Rad Laboratories                   1
                       Catalent                               1
                       IDEXX Laboratories                     1
                       Illumina Inc                           1
Industrials            Huntington Ingalls Industries          1
                       IHS Markit Ltd.                        1
                       United Rentals, Inc.                   1
Information Technology ANSYS                                  1
                       Applied Materials Inc.                 1
                       Cadence Design Systems                 1
                       Hewlett Packard Enterprise             1
                       Intuit Inc.                            1
                       KLA Corporation                        1
                       Keysight Technologies                  1
                       PayPal                                 1
                       Synopsys Inc.                          1
                       Tyler Technologies                     1
                       Western Union Co                       1
                       Zebra Technologies                     1
Materials              Martin Marietta Materials              1
Real Estate            AvalonBay Communities                  1
                       CBRE Group                             1
                       Federal Realty Investment Trust        1
                       Kimco Realty                           1
                       Regency Centers Corporation            1
                       Simon Property Group Inc               1
                       UDR, Inc.                              1
                       Welltower Inc.                         1

In [19]:
df[df.goldenBuySignal > 0 ].groupby(['GICS Sector', 'Security']).agg({'totalBuy':['max']})

,,totalBuy
,,max
GICS Sector,Security,
Information Technology,Intel Corp.,1


In [20]:
df[df.rsiBuySignal > 0 ].groupby(['GICS Sector', 'Security']).agg({'totalBuy':['max']})

totalBuy
                                                          max
GICS Sector            Security                              
Communication Services T-Mobile US                          1
                       Verizon Communications               1
Consumer Discretionary Advance Auto Parts                   1
                       AutoZone Inc                         1
                       Newell Brands                        1
                       O'Reilly Automotive                  1
                       Pool Corporation                     1
                       Royal Caribbean Group                1
                       VF Corporation                       1
Consumer Staples       Molson Coors Beverage Company        1
                       PepsiCo Inc.                         1
                       Procter & Gamble                     1
                       The Clorox Company                   1
Financials             Allstate Corp                        1
                       Assurant                             1
                       Intercontinental Exchange            1
Health Care            Bristol-Myers Squibb                 1
                       Cardinal Health Inc.                 1
                       Cerner                               1
                       DaVita Inc.                          1
                       Incyte                               1
                       Merck & Co.                          1
                       Regeneron Pharmaceuticals            1
                       ResMed                               1
Industrials            Carrier Global                       1
                       Pentair plc                          1
                       Roper Technologies                   1
Information Technology Akamai Technologies Inc              1
                       Jack Henry & Associates              1
Materials              DuPont de Nemours Inc                1
                       FMC Corporation                      1
Real Estate            American Tower Corp.                 1
                       Digital Realty Trust Inc             1
                       Equinix                              1
                       SBA Communications                   1
Utilities              Alliant Energy Corp                  1
                       American Electric Power              1
                       Dominion Energy                      1
                       Duke Energy                          1
                       Eversource Energy                    1
                       Exelon Corp.                         1
                       WEC Energy Group                     1
                       Xcel Energy Inc                      1

In [21]:
df[(df.totalBuy > 1)].groupby(['GICS Sector', 'Security']).agg({'totalBuy':['max']})

,,totalBuy
,,max
GICS Sector,Security,
